In [1]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
import os
import pickle
%matplotlib inline
from datetime import datetime
from dateutil.parser import parse

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
with open("apartment_sale.pkl", 'rb') as picklefile: 
    zillow_apartment_df = pickle.load(picklefile)
zillow_apartment_df.head()

,RegionName,City,State,Metro,CountyName,SizeRank,BedSize,Year,Month,Amount
70,98103,Seattle,WA,Seattle,King,71,Studio,2017,01,1457.0
70,98103,Seattle,WA,Seattle,King,71,Studio,2017,02,1482.5
70,98103,Seattle,WA,Seattle,King,71,Studio,2017,03,1275.0
70,98103,Seattle,WA,Seattle,King,71,Studio,2017,04,1385.0
70,98103,Seattle,WA,Seattle,King,71,Studio,2017,05,1595.0


In [3]:
zillow_apartment_df.shape

(1047, 10)

In [4]:
with open("google_trend.pkl", 'rb') as picklefile: 
    google_trend_df = pickle.load(picklefile)

google_trend_df.head()
k = google_trend_df.groupby(['Year', 'Month']).count()


In [5]:
google_trend_df ['Date'] = pd.to_datetime((google_trend_df['Year'] + '-' + google_trend_df['Month']+ '-01') ,format='%Y-%m-%d')
google_trend_df.shape

(516, 5)

In [6]:

date_shift_six_month_df = google_trend_df.Date+   pd.offsets.DateOffset(months=-6)
google_trend_df ['Shift_Year'] = date_shift_six_month_df.dt.year
google_trend_df ['Shift_Month'] = date_shift_six_month_df.dt.month.map("{:02}".format)
google_trend_df.shape

(516, 7)

In [7]:
google_trend_df_SeattleRent = google_trend_df.loc[google_trend_df['Term'] == 'SeattleRent']
google_trend_df_SeattleRent.rename(columns={"Amount": "Search_Count_SeattleRent"}, inplace=True)
google_trend_df_SeattleRent = google_trend_df_SeattleRent.drop(['Term', 'Year','Month', 'Date'], 1)

google_trend_df_SeattleHousing = google_trend_df.loc[google_trend_df['Term'] == 'SeattleHousing']
google_trend_df_SeattleHousing.rename(columns={"Amount": "Search_Count_SeattleHousing"}, inplace=True)
google_trend_df_SeattleHousing = google_trend_df_SeattleHousing.drop(['Term', 'Year','Month', 'Date'], 1)

google_trend_df_seattleApartment = google_trend_df.loc[google_trend_df['Term'] == 'seattleApartment']
google_trend_df_seattleApartment.rename(columns={"Amount": "Search_Count_seattleApartment"}, inplace=True)
google_trend_df_seattleApartment = google_trend_df_seattleApartment.drop(['Term', 'Year','Month', 'Date'], 1)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [8]:
google_trend_df_SeattleRent.tail()

,Search_Count_SeattleRent,Shift_Year,Shift_Month
167,31,2017,06
168,39,2017,07
169,38,2017,08
170,38,2017,09
171,40,2017,10


In [9]:
with open("zillow_housing.pkl", 'rb') as picklefile: 
    zillow_housing_df = pickle.load(picklefile)
zillow_housing_df.head()

,RegionID,RegionName,StateName,SizeRank,Year,Month,Amount
17,16037.0,Seattle,Washington,18,2008,03,421300.0
17,16037.0,Seattle,Washington,18,2008,04,424000.0
17,16037.0,Seattle,Washington,18,2008,05,426000.0
17,16037.0,Seattle,Washington,18,2008,06,422900.0
17,16037.0,Seattle,Washington,18,2008,07,414400.0


In [10]:
zillow_housing_df ['Date'] = pd.to_datetime((zillow_housing_df['Year'] + '-' + zillow_housing_df['Month']+ '-01') ,format='%Y-%m-%d')
zillow_housing_df_shift_six_month_df = zillow_housing_df.Date+   pd.offsets.DateOffset(months=-6)
zillow_housing_df ['Shift_Year'] = zillow_housing_df_shift_six_month_df.dt.year.astype(str)
zillow_housing_df ['Shift_Month'] = zillow_housing_df_shift_six_month_df.dt.month.map("{:02}".format)
zillow_housing_df = zillow_housing_df.drop(['RegionID','RegionName','StateName','SizeRank','Year','Month', 'Date'], 1)


zillow_housing_df.head()

,Amount,Shift_Year,Shift_Month
17,421300.0,2007,09
17,424000.0,2007,10
17,426000.0,2007,11
17,422900.0,2007,12
17,414400.0,2008,01


In [11]:
zillow_apartment_df['Shift_Year'] = zillow_apartment_df['Year'].astype(str)
zillow_apartment_df['Shift_Month'] = zillow_apartment_df['Month'].astype(str)

In [12]:
merge_housing = pd.merge(zillow_apartment_df, zillow_housing_df, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing = merge_housing.rename(columns={'Amount_y':'Zillow_Housing_price'})
merge_housing = merge_housing.rename(columns={'Amount_x':'Apartment_rent'})


merge_housing.shape

(1047, 13)

In [13]:
google_trend_df_SeattleRent['Shift_Year'] = google_trend_df_SeattleRent['Shift_Year'].astype(str)
google_trend_df_SeattleRent['Shift_Month'] = google_trend_df_SeattleRent['Shift_Month'].astype(str)
k = google_trend_df_SeattleRent.groupby(['Shift_Year', 'Shift_Month']).count()
k.head(1)


,,Search_Count_SeattleRent
Shift_Year,Shift_Month,
2003,07,1


In [14]:
merge_housing = merge_housing.drop(['RegionName','City','State','Metro', 'CountyName', 'SizeRank'], 1)

merge_housing  = pd.merge(merge_housing, google_trend_df_SeattleRent, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing.shape

(1047, 8)

In [51]:

google_trend_df_SeattleHousing['Shift_Year'] = google_trend_df_SeattleHousing['Shift_Year'].astype(str)
google_trend_df_SeattleHousing['Shift_Month'] = google_trend_df_SeattleHousing['Shift_Month'].astype(str)

merge_housing  = pd.merge(merge_housing, google_trend_df_SeattleHousing, on=['Shift_Year', 'Shift_Month'], how='left')

google_trend_df_seattleApartment['Shift_Year'] = google_trend_df_seattleApartment['Shift_Year'].astype(str)
google_trend_df_seattleApartment['Shift_Month'] = google_trend_df_seattleApartment['Shift_Month'].astype(str)

merge_housing  = pd.merge(merge_housing, google_trend_df_seattleApartment, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing_x,Search_Count_seattleApartment_x,Search_Count_SeattleHousing_y,Search_Count_seattleApartment_y
0,Studio,2017,01,1457.0,2017,01,654100.0,49,31,28,31,28
1,Studio,2017,02,1482.5,2017,02,667000.0,43,30,26,30,26
2,Studio,2017,03,1275.0,2017,03,674400.0,43,27,23,27,23
3,Studio,2017,04,1385.0,2017,04,671400.0,35,27,24,27,24
4,Studio,2017,05,1595.0,2017,05,676000.0,33,25,21,25,21


In [ ]:
redfin_housing=pd.read_csv('data/House/Redfin_Seattle_All.csv')
redfin_housing['Region Type'].value_counts()
redfin_housing.head()

In [ ]:
redfin_housing = redfin_housing.drop(['City','Measure Display','Period End','State','State Code', 'Table Id', 'Property Type','Period Duration', 'Region Type', 'Region', 'Worksheet Filter'],1)
redfin_housing.head()

In [ ]:
redfin_housing ['Date'] = pd.to_datetime((redfin_housing['Period Begin']) ,format='%m/%d/%Y')
redfin_housing_shif = redfin_housing.Date+   pd.offsets.DateOffset(months=-6)
redfin_housing ['Shift_Year'] = redfin_housing_shif.dt.year.astype(str)
redfin_housing ['Shift_Month'] = redfin_housing_shif.dt.month.map("{:02}".format)
redfin_housing = redfin_housing.drop(['Period Begin'], 1)


redfin_housing.head()

In [55]:
merge_housing  = pd.merge(merge_housing, redfin_housing, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing_x,Search_Count_seattleApartment_x,...,New Listings Mom,New Listings Yoy,Number of Records,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy,Date
0,Studio,2017,01,1457.0,2017,01,654100.0,49,31,28,...,-12.00%,-8.10%,1.0,0.245077,-0.009731,-0.005338,0.595837,-0.055433,0.085781,2017-07-01
1,Studio,2017,02,1482.5,2017,02,667000.0,43,30,26,...,-6.00%,-5.50%,1.0,0.289786,0.044710,0.002615,0.581190,-0.014647,0.124437,2017-08-01
2,Studio,2017,03,1275.0,2017,03,674400.0,43,27,23,...,12.30%,-6.20%,1.0,0.258972,-0.030814,-0.036182,0.556420,-0.024769,0.127969,2017-09-01
3,Studio,2017,04,1385.0,2017,04,671400.0,35,27,24,...,-20.40%,11.30%,1.0,0.278618,0.019646,-0.058369,0.552363,-0.004057,0.106503,2017-10-01
4,Studio,2017,05,1595.0,2017,05,676000.0,33,25,21,...,-31.90%,12.10%,1.0,0.310937,0.032320,0.012824,0.501499,-0.050865,0.096896,2017-11-01


In [56]:
with open('all_data.pkl', 'wb') as picklefile:
    pickle.dump(merge_housing, picklefile)

In [57]:
with open("all_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)
my_old_data.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing_x,Search_Count_seattleApartment_x,...,New Listings Mom,New Listings Yoy,Number of Records,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy,Date
0,Studio,2017,01,1457.0,2017,01,654100.0,49,31,28,...,-12.00%,-8.10%,1.0,0.245077,-0.009731,-0.005338,0.595837,-0.055433,0.085781,2017-07-01
1,Studio,2017,02,1482.5,2017,02,667000.0,43,30,26,...,-6.00%,-5.50%,1.0,0.289786,0.044710,0.002615,0.581190,-0.014647,0.124437,2017-08-01
2,Studio,2017,03,1275.0,2017,03,674400.0,43,27,23,...,12.30%,-6.20%,1.0,0.258972,-0.030814,-0.036182,0.556420,-0.024769,0.127969,2017-09-01
3,Studio,2017,04,1385.0,2017,04,671400.0,35,27,24,...,-20.40%,11.30%,1.0,0.278618,0.019646,-0.058369,0.552363,-0.004057,0.106503,2017-10-01
4,Studio,2017,05,1595.0,2017,05,676000.0,33,25,21,...,-31.90%,12.10%,1.0,0.310937,0.032320,0.012824,0.501499,-0.050865,0.096896,2017-11-01
